In [1]:
import tensorflow as tf

2022-08-28 01:10:45.286838: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 01:10:45.286875: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(train_images, train_labels), (test_images, test_labels) =tf.keras.datasets.mnist.load_data()
print(train_images.shape,train_labels.shape)
print(len(train_images),len(test_labels))
sample = test_images[0]
train_images = train_images.reshape((len(train_images),28,28,1))
test_images = test_images.reshape((len(test_images),28,28,1))

train_images,test_images = train_images.astype("float32"),test_images.astype("float32")
print(train_images.shape,test_images.shape)
print(type(train_labels[0]))
print(sample.shape)
buf = tf.image.encode_png(sample[:, :, tf.newaxis])
tf.io.write_file("cnn/mnist_savedmodel/sample.png", buf)

(60000, 28, 28) (60000,)
60000 10000
(60000, 28, 28, 1) (10000, 28, 28, 1)
<class 'numpy.uint8'>
(28, 28)


2022-08-28 01:10:49.484668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-28 01:10:49.484699: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-28 01:10:49.484720: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (morio-H87M-S01): /proc/driver/nvidia/version does not exist
2022-08-28 01:10:49.488191: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1))
    self.maxpooling1 = tf.keras.layers.MaxPool2D((2,2))
    self.conv2 = tf.keras.layers.Conv2D(64,(3,3),activation='relu')
    self.maxpooling2 = tf.keras.layers.MaxPool2D((2,2))
    self.conv3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu')
    self.flatten = tf.keras.layers.Flatten()
    self.dens1 = tf.keras.layers.Dense(64,activation='relu')
    self.dens2 = tf.keras.layers.Dense(10,activation='softmax')
  
  def call(self,x):
    x = self.conv1(x)
    x = self.maxpooling1(x)
    x = self.conv2(x)
    x = self.maxpooling2(x)
    x = self.conv3(x)
    x = self.flatten(x)
    x = self.dens1(x)
    return self.dens2(x)
model = CNN()
print(model.variables)
print(model.trainable_variables)

[]
[]


In [4]:
""" model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5) """

" model.compile(optimizer='adam',\n              loss='sparse_categorical_crossentropy',\n              metrics=['accuracy'])\n\nmodel.fit(train_images, train_labels, epochs=5) "

In [5]:
#損失関数
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
#最適化関数
optimizer = tf.keras.optimizers.Adam()
# 評価指標
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x,y):
  with tf.GradientTape() as tape:
    predictions = model(x, training=True)
    loss = loss_object(y, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y, predictions)

@tf.function
def test_step(x, t):
    test_predictions = model(x)
    t_loss = loss_object(t, test_predictions)

    test_loss(t_loss)
    test_accuracy(t, test_predictions)


In [9]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(16, 28, 28, 1)


In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)
print(train_ds)
EPOCHS = 10
""" for epoch in range(EPOCHS):
  train_step(train_images,train_labels)
 """
for epoch in range(EPOCHS):
  for images, labels in train_ds:
    #print(images.shape)
    #print(labels.shape)
    train_step(images, labels) #学習

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels) #評価

  template = 'Epoch {}, Loss: {}, Accuracy: {}, test-Loss: {}, test-Accuracy:{}'
  print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result()*100))

2022-08-26 21:53:24.798596: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.
2022-08-26 21:53:24.915677: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 31360000 exceeds 10% of free system memory.
2022-08-26 21:53:24.941043: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.


<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.uint8)>


2022-08-26 21:53:25.454628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-26 21:53:49.973307: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 31360000 exceeds 10% of free system memory.


Epoch 1, Loss: 0.2274152636528015, Accuracy: 94.82333374023438, test-Loss: 0.07201920449733734, test-Accuracy:97.73999786376953


2022-08-26 21:53:51.316140: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 2, Loss: 0.14609336853027344, Accuracy: 96.43916320800781, test-Loss: 0.06203008443117142, test-Accuracy:98.04000091552734
Epoch 3, Loss: 0.11503647267818451, Accuracy: 97.08333587646484, test-Loss: 0.06337379664182663, test-Accuracy:97.98332977294922
Epoch 4, Loss: 0.09756989777088165, Accuracy: 97.47874450683594, test-Loss: 0.05750598385930061, test-Accuracy:98.17500305175781
Epoch 5, Loss: 0.08602694422006607, Accuracy: 97.74066925048828, test-Loss: 0.05848097801208496, test-Accuracy:98.16999816894531
Epoch 6, Loss: 0.07724744826555252, Accuracy: 97.9463882446289, test-Loss: 0.05675932765007019, test-Accuracy:98.2550048828125
Epoch 7, Loss: 0.07077979296445847, Accuracy: 98.10452270507812, test-Loss: 0.05447050929069519, test-Accuracy:98.34285736083984
Epoch 8, Loss: 0.06527946144342422, Accuracy: 98.23854064941406, test-Loss: 0.05328453704714775, test-Accuracy:98.41124725341797
Epoch 9, Loss: 0.06079905107617378, Accuracy: 98.35610961914062, test-Loss: 0.05280643701553345, te

In [12]:
print(train_images.shape)
print(type(train_labels))

(60000, 28, 28, 1)
<class 'numpy.ndarray'>


In [3]:
#確立として解釈できるように、ソフトマックスを介して出力タイプを変換
inputs = tf.keras.Input((28,28,1),name="input",dtype=tf.float32)
outputs = model(inputs)
#print(x)
#outputs = tf.keras.layers.Dense(10,name="output")(x)
#print(outputs)
#outputs = tf.keras.layers.Softmax(name="output")(x)
probability_model = tf.keras.Model(inputs=inputs, outputs=outputs)
print(test_images[:1, :, :][0].shape)
with open("./cnn/expected_values.txt", "w") as f:
    values = probability_model(test_images[1:2, :, :])[0].numpy()
    print(*values, sep=", ", file=f)

NameError: name 'model' is not defined

In [8]:
directory = "cnn/mnist_savedmodel/"
tf.saved_model.save(probability_model, directory)

2022-08-26 21:57:39.906150: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: cnn/mnist_savedmodel/assets
